# Welcome to Core Session 4

## The 3 themes for today:

1. Using MCP servers
2. Writing MCP servers
3. Context Engineering

# Agenda

1. Intro to MCP
 -  Recap & Cookbook
 -  MCP introduction
 -  Hands on - use an MCP server

2. Make your own MCP server
 - The database code
 - The skeleton
 - The exercise

3. Digital Twin reprise
 - Context engineering with MCP
 - Back to your teams!

### But first..


# Lab 1 - The Cookbook

In [ ]:
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display, update_display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)

## Recipe for a simple request to gpt-5-nano

In [ ]:
agent = Agent("Joker", model="gpt-5-nano")
result = await Runner.run(agent, "Please tell a joke about AI Agents")
result.final_output

In [ ]:
# Please tell me how to find the integral of 2x dx


## Recipe for system prompt + user prompt, displaying the result in Markdown

In [ ]:
instructions = "You tell jokes by giving a standup routine, like a comedian putting on a show."
agent = Agent(name="Joker", instructions=instructions, model="gpt-4.1-mini")
result = await Runner.run(agent, "Tell a joke about AI Agents")
display(Markdown(result.final_output))

## And streaming back results

In [ ]:
result = Runner.run_streamed(agent, input="Tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

## Recipe for streaming back the results in Markdown

In [ ]:
reply = "## Jokester\n"
display_id = display(Markdown(reply), display_id=True).display_id
result = Runner.run_streamed(agent, input="Please tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        reply += event.data.delta
        update_display(Markdown(reply), display_id=display_id)


## Recipe for a Simple Tool

In [ ]:
@function_tool
def write_to_file(contents: str) -> str:
    """ Write the given contents to a local file called output.md 

    Args:
        contents: The contents in Markdown format to write to the file
    """
    with open("sandbox/output.md", "w") as f:
        f.write(contents)
    return "File written successfully"

In [ ]:
agent = Agent("Joker", instructions=instructions, tools=[write_to_file])
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output

## Recipe for Structured Outputs

In [ ]:
class Joke(BaseModel):
    opening_patter: str = Field("Some preamble to get the crowd in the mood for the joke")
    setup: str = Field("The setup of the joke")
    punchline: str = Field("The punchline of the joke")

In [ ]:
agent = Agent("Joker", tools=[write_to_file], output_type=Joke)
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output_as(Joke)